In [7]:
# Get the samples into a bit slicer based on observed amplitude
from gnuradio import blocks
from gnuradio import digital
from gnuradio import eng_notation
from gnuradio import filter
from gnuradio import gr
from gnuradio.eng_option import eng_option
from gnuradio.filter import firdes
from optparse import OptionParser
import pmt
import sip
import sys
from bitarray import bitarray
import time
from code import mybits

ImportError: cannot import name mybits

In [ ]:
#####
# file source to string bit output
# ook_rx.run(duration=2)
#####
class ook_rx_from_file(gr.top_block):

    def __init__(self, file='/home/jumson/Desktop/share/pinode_gitlab/old_samps/ook_c313M_s8M.iq'):
        gr.top_block.__init__(self, "Ook Rx To Vector")

        ##################################################
        # Variables
        ##################################################
        self.working_samp_rate = working_samp_rate = 400e3
        self.threshold = threshold = 500
        self.samp_rate = samp_rate = 8e6
        self.freq = freq = 315e6
        self.filter_transition = filter_transition = 1e3
        self.filter_cutoff = filter_cutoff = 50e3
        self.center_freq = center_freq = 313e6
        self.file_name = file_name = file

        ##################################################
        # Blocks
        ##################################################
        self.freq_xlating_fir_filter_xxx_0 = filter.freq_xlating_fir_filter_ccc(int(samp_rate/working_samp_rate), (firdes.low_pass(1, samp_rate, filter_cutoff, filter_transition)), freq-center_freq, samp_rate)
        self.digital_binary_slicer_fb_0 = digital.binary_slicer_fb()
        self.blocks_vector_sink_x_0_0 = blocks.vector_sink_b(1, 1024)
        self.blocks_throttle_0 = blocks.throttle(gr.sizeof_gr_complex*1, samp_rate,True)
        self.blocks_file_source_0 = blocks.file_source(gr.sizeof_gr_complex*1, file_name, True)
        self.blocks_file_source_0.set_begin_tag(pmt.PMT_NIL)
        self.blocks_complex_to_mag_0 = blocks.complex_to_mag(1)
        self.blocks_add_const_vxx_0 = blocks.add_const_vff((-1*threshold, ))



        ##################################################
        # Connections
        ##################################################
        self.connect((self.blocks_add_const_vxx_0, 0), (self.digital_binary_slicer_fb_0, 0))
        self.connect((self.blocks_complex_to_mag_0, 0), (self.blocks_add_const_vxx_0, 0))
        self.connect((self.blocks_file_source_0, 0), (self.blocks_throttle_0, 0))
        self.connect((self.blocks_throttle_0, 0), (self.freq_xlating_fir_filter_xxx_0, 0))
        self.connect((self.digital_binary_slicer_fb_0, 0), (self.blocks_vector_sink_x_0_0, 0))
        self.connect((self.freq_xlating_fir_filter_xxx_0, 0), (self.blocks_complex_to_mag_0, 0))

    def get_working_samp_rate(self):
        return self.working_samp_rate

    def set_working_samp_rate(self, working_samp_rate):
        self.working_samp_rate = working_samp_rate

    def get_threshold(self):
        return self.threshold

    def set_threshold(self, threshold):
        self.threshold = threshold
        self.blocks_add_const_vxx_0.set_k((-1*self.threshold, ))

    def get_samp_rate(self):
        return self.samp_rate

    def set_samp_rate(self, samp_rate):
        self.samp_rate = samp_rate
        self.freq_xlating_fir_filter_xxx_0.set_taps((firdes.low_pass(1, self.samp_rate, self.filter_cutoff, self.filter_transition)))
        self.blocks_throttle_0.set_sample_rate(self.samp_rate)

    def get_freq(self):
        return self.freq

    def set_freq(self, freq):
        self.freq = freq
        self.freq_xlating_fir_filter_xxx_0.set_center_freq(self.freq-self.center_freq)

    def get_filter_transition(self):
        return self.filter_transition

    def set_filter_transition(self, filter_transition):
        self.filter_transition = filter_transition
        self.freq_xlating_fir_filter_xxx_0.set_taps((firdes.low_pass(1, self.samp_rate, self.filter_cutoff, self.filter_transition)))

    def get_filter_cutoff(self):
        return self.filter_cutoff

    def set_filter_cutoff(self, filter_cutoff):
        self.filter_cutoff = filter_cutoff
        self.freq_xlating_fir_filter_xxx_0.set_taps((firdes.low_pass(1, self.samp_rate, self.filter_cutoff, self.filter_transition)))

    def get_center_freq(self):
        return self.center_freq

    def set_center_freq(self, center_freq):
        self.center_freq = center_freq
        self.freq_xlating_fir_filter_xxx_0.set_center_freq(self.freq-self.center_freq)

    def get_file_name(self):
        return self.file_name

    def get_bitarray(self):
        dat = self.blocks_vector_sink_x_0_0.data()
        b = bitarray()
        for samp in dat:
            if samp == 1:
                b.append(True)
            if samp == 0:
                b.append(False)
        return b

    # operates on the get_bitarray data
    def get_bursts(self):
        return mybits.bursts(self.get_bitarray())

    # simple run to read the data in
    def srun(self, duration=2):
        try:
            # run for a designated time
            self.start()
            t_end = time.time() + duration
            while time.time() < t_end:
                pass
        except EOFError:
            pass
        self.stop()
        self.wait()
        return self.get_bitarray()

    # one stop shop for rx to return most probable
    # bits as string
    def run(self, duration=1):
        try:
            # run for a designated time
            self.start()
            t_end = time.time() + duration
            while time.time() < t_end:
                pass
        except EOFError:
            pass
        self.stop()
        self.wait()
        rxout = self.get_bursts()
        bits = mybits.analyze_bursts(rxout)
        return str(bits)

    # maintains the freq while shifting the threshold for bits for the slicer
    # call: rx.auto_thresh(freq=rx.get_freq())
    def auto_thresh(self,duration=1,step=5,max_thresh=100,min_thresh=-100,freq=315e6):
        thresh = min_thresh

        found_true = 0
        found_false = 0
        true_bits = 0
        false_bits = 0
        Done = True
        while thresh != max_thresh:
            # this is to clean out the data()?
            self.__init__()
            self.set_freq(freq)
            thresh += step
            self.set_threshold(thresh)
            cur_b = self.srun(duration)
            print "threshold:", thresh, "size:", cur_b.length(), "true:",true_bits,"false:",false_bits
            try:
                true_bits = cur_b.index(True)
                found_true = thresh
            except:
                found_true = -1
            try:
                false_bits = cur_b.index(False)
                found_false = thresh
            except:
                found_false = -1

            if (found_true > 0) and (found_false > 0):
                print "found true at", found_true
                print "found false at", found_false
                # this mens there are some differentiating bits

    def get_stats(self):
        print "     working_samp_rate =", self.get_working_samp_rate()
        print "     threshold =", self.get_threshold()
        print "     samp_rate =", self.get_samp_rate()
        print "     freq =", self.get_freq()
        print "     filter_transition =", self.get_filter_transition()
        print "     filter_cutoff =", self.get_filter_cutoff()
        print "     center_freq =", self.get_center_freq()
        print "      file_name =", self.get_file_name()


    def get_data(self):
        return self.blocks_vector_sink_x_0_0.data()
